##Setup

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [ ]:
problem = """
<GOAL>ontable b1, clear b1, ontable b2, clear b2, ontable b3, clear b3, ontable b4, clear b4
<INIT>handempty, ontable b1, on b2 b3, clear b2, ontable b3, on b4 b1, clear b4
<ACTION> pick-up <PRE> clear x, ontable x, handempty <EFFECT> not ontable x, not clear x, not handempty, holding x
<ACTION> put-down <PRE> holding x <EFFECT> not holding x, clear x, handempty, ontable x
<ACTION> stack <PRE> holding x, clear y <EFFECT> not holding x, not clear y, clear x, handempty, on x y
<ACTION> unstack <PRE> on x y, clear x, handempty <EFFECT> holding x, clear y, not clear x, not handempty, not on x y"
"""

In [ ]:
predicted_plan = "unstack b2 b3, put-down b2, unstack b4 b1, put-down b4"

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
# Downloading Plansformer from Dropbox
!wget -O plansformer.zip https://www.dropbox.com/scl/fi/sc2syaax93i50bn14gn3w/plansformer_v3.zip?rlkey=a5qf2fanfn5028yx78drf1hic&dl=0
!unzip plansformer.zip -d ./
model_path = "codet5-base/model_files"
tokenizer = RobertaTokenizer.from_pretrained(model_path, local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True)

# Encode inputs
sentence_a = problem
sentence_b = predicted_plan

encoder_input_ids = tokenizer(sentence_a, return_tensors="pt", add_special_tokens=True).input_ids
with tokenizer.as_target_tokenizer():
    decoder_input_ids = tokenizer(sentence_b, return_tensors="pt", add_special_tokens=True).input_ids

outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids, output_attentions=True)

encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

--2024-07-24 13:26:37--  https://www.dropbox.com/scl/fi/sc2syaax93i50bn14gn3w/plansformer_v3.zip?rlkey=a5qf2fanfn5028yx78drf1hic
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfaf43f85224086652394230775.dl.dropboxusercontent.com/cd/0/inline/CXXE9AfQ39L2EcoKPuAn17ZPvnxTw-nc7rN6J1F7UK-_eWvcc7KXGOvoIgCxL6nbv_Xs57WiI0UJZRN6zBn4gDImgc4dsDhDITltV_VBdCJNhEeCpOWxyQv1fMY1oIefvHb58AqnzCNE9vQrNTgk115f/file# [following]
--2024-07-24 13:26:38--  https://ucfaf43f85224086652394230775.dl.dropboxusercontent.com/cd/0/inline/CXXE9AfQ39L2EcoKPuAn17ZPvnxTw-nc7rN6J1F7UK-_eWvcc7KXGOvoIgCxL6nbv_Xs57WiI0UJZRN6zBn4gDImgc4dsDhDITltV_VBdCJNhEeCpOWxyQv1fMY1oIefvHb58AqnzCNE9vQrNTgk115f/file
Resolving ucfaf43f85224086652394230775.dl.dropboxusercontent.com (ucfaf43f85224086652394230775.dl.dropboxusercontent.com)... 162.125.65.1

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


##Model_view

In [ ]:
from bertviz import model_view
for layer in range(0, 12):  # Change this to the number of layers you want to visualize
    html_model_view = model_view(
        encoder_attention = outputs.encoder_attentions,
        decoder_attention = outputs.decoder_attentions,
        cross_attention = outputs.cross_attentions,
        encoder_tokens = encoder_text,
        decoder_tokens = decoder_text,
        include_layers=[layer],
        html_action='return',

    )
    display(html_model_view)

    # Save the HTML content to a file in the current working directory
    file_path = f"/content/model_view_layer_{layer}.html"
    try:
        with open(file_path, 'w') as file:
            file.write(html_model_view.data)
        print(f"HTML file saved successfully for layer {layer}")
    except Exception as e:
        print(f"Error saving HTML file for layer {layer}: {e}")

##Head_view

In [ ]:
from bertviz import head_view
head_view(outputs.encoder_attentions, encoder_text, html_action='return')

file_path = f"/content/head_view.html"
try:
    with open(file_path, 'w') as file:
         file.write(html_model_view.data)
    print(f"HTML file saved successfully")
except Exception as e:
    print(f"Error saving HTML file: {e}")

HTML file saved successfully
